# Logistic Regression (Binary class)

In [91]:
# Python 2, 3 compatible
from __future__ import print_function, division

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import pandas as pd

In [80]:
# Datasets, data loading
from torchvision import datasets, transforms
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [42]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [82]:
# Globals
RSEED = 100

In [54]:
ls data/

diabetes.csv.gz     dogscats.zip        names_train.csv.gz
dogscats/           names_test.csv.gz   shakespeare.txt.gz


## MNIST dataset

In [57]:
#MNIST dataset
train_data = datasets.MNIST(root='./data', 
                      train=True,
                      transform=transforms.ToTensor(),
                      download=True)
test_data = datasets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor())

In [60]:
# Training setting 
batch_size = 64

In [62]:
# Data loader (Input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                         batch_size=batch_size,
                                         shuffle=False)


## Breast cancer (binary classification) dataset

In [85]:
cancer_data = load_breast_cancer()
data = cancer_data.data; #print(data[:10,-5:])
target = cancer_data.target; #print(target[-5:])
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size=0.33, random_state=RSEED)

In [90]:
print(X_train.shape, X_test.shape)

(381, 30) (188, 30)


## 1. Linear -> Sigmoid -> BCELoss(yhat_prob, y_prob)

In [38]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = torch.nn.Linear(3,1) 
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self,x):
        """x is a 2D tensor of size (mini_batch, d)
        If mini_batch size is 1, x can be a 1D tensor of length d.
        Returns a forwarded tensor object"""
        z = self.linear(x)
        y_hat = self.sigmoid(z)
        return y_hat

criterion = torch.nn.BCELoss(size_average=False)        

In [39]:
m = Model()
print("Initial weights: ", m.linear.weight)
print("Initial bias: ", m.linear.bias)

# Single input and a target
x = torch.tensor([10,1,1], dtype=torch.float, requires_grad=True)
y = torch.tensor([1],dtype=torch.float)
y_hat = m.forward(x)

l = criterion(y_hat, y)
print (l)

Initial weights:  Parameter containing:
tensor([[ 0.5702, -0.2028,  0.1714]])
Initial bias:  Parameter containing:
tensor([ 0.2596])
tensor(1.00000e-03 *
       2.6533)


In [25]:
#check if loss is as expected by binary cross entropy 
print("class: ", y.item())
print("predicted probability of class 1: ", y_hat)
print(torch.log(y_hat).item())

class:  1.0
predicted probability of class 1:  tensor([ 0.9526])
-0.048508379608392715


In [40]:
print("Before:  ", x.grad)
#backprop
l.backward()

#dl/dinput
print("After: ", x.grad)
x.grad = None

Before:   None
After:  tensor(1.00000e-03 *
       [-1.5110,  0.5374, -0.4540])


Test with breast data

## 2. Linear -> BCEWWithLogitsLoss(z, y_prob)
